# 🔴 PART 1: CORE DESIGN


### **1. End-to-End LLM Feature Design**

**Quick Framework (use for ANY design question):**

```
1. Clarify → 2. Simple baseline → 3. Where it breaks → 4. How to improve
```

**Example: Customer Support Chatbot**

**Clarify:**
- Volume? (100 tickets/day vs 10K/day)
- Latency requirement? (<2s for chat)
- Knowledge base? (docs, past tickets, FAQs)
- Budget constraints?

**Baseline Design:**
```
User Query → Embedding → Vector Search (top-3 chunks)
→ LLM (GPT-4/Claude) with context → Response
```

**Why RAG vs Fine-tuning?**
- **RAG when:** Knowledge changes frequently, need citations, limited budget
- **Fine-tuning when:** Specific tone/style needed, repeated patterns, cost matters long-term

**Caching strategy:**
- Cache embeddings (they don't change)
- Cache common queries at response level
- Don't cache final LLM calls (context varies)

**Evaluation:**
- Offline: retrieval precision@k, answer relevance (LLM-as-judge)
- Online: thumbs up/down, resolution rate, human spot-checks

**Cost control:**
- Smaller model for routing/classification
- Larger model only for complex queries
- Prompt compression, context pruning

---

### **2. RAG-Specific Debugging**

#### **"Answers are confident but wrong"**

**Debug checklist:**
1. **Check retrieval first** → Are right docs retrieved? (log top-3 chunks)
2. **Context position** → Move most relevant chunk to end (recency bias)
3. **Chunk quality** → Too small = missing context, too large = noise
4. **Prompt instruction** → Add "Only use provided context, admit if unsure"

#### **"Retrieval correct, generation bad"**

- **Model ignoring context** → Use stronger instruction, add examples (few-shot)
- **Context too long** → Model lost in middle, use re-ranking or summarization first
- **Conflicting info in context** → Pre-filter contradictory chunks

#### **"Larger chunks help recall but hurt quality"**

**Solution hierarchy:**
1. **Hybrid chunking** → Keep small chunks, but retrieve parent chunks for context
2. **Two-stage retrieval** → Small chunks for search, expand window for generation
3. **Sliding window overlap** → 20-30% overlap between chunks

---

### **3. Model Choice & Adaptation**

#### **Open-source vs API**

| Factor | Open-source | API |
|--------|-------------|-----|
| Cost | High upfront, cheap at scale | Pay-per-token |
| Control | Full (prompts, weights) | Limited |
| Latency | You control infra | Network dependent |
| Compliance | Keep data internal | Data leaves premises |

**Startup default:** Start with API (GPT-4/Claude), switch to open-source when cost > $5K/month

#### **LoRA vs Full Fine-tuning**

**Use LoRA when:**
- Limited data (<10K examples)
- Need multiple task-specific models
- Fast iteration needed

**Use Full FT when:**
- Domain shift is massive (legal → medical)
- Have 50K+ high-quality examples
- Model architecture change needed

**Avoid PEFT when:**
- Base model is already very small (<3B params)
- Task needs architectural changes (new tokens, special layers)

---

### **4. Evaluation & Metrics**

#### **"Loss decreasing but quality worse"**

**Common causes:**
1. **Train-test mismatch** → Training on clean data, users give messy input
2. **Overfitting to metric** → Model gaming the loss function
3. **Proxy metric misalignment** → Optimizing BLEU but users want factuality

**Fix:** Add hold-out set from real production data, use human eval

#### **"How to evaluate GenAI before launch?"**

**3-tier system:**

1. **Automated (fast, cheap)**
   - Retrieval: precision@3, MRR
   - Generation: perplexity, length, refusal rate
   - LLM-as-judge for relevance (GPT-4 rating 1-5)

2. **Human spot-check (medium)**
   - 100-200 examples weekly
   - Domain expert review for factuality

3. **Production metrics (slow, real)**
   - User engagement (thumbs, session length)
   - Task success (ticket resolved? query answered?)

**Preventing regression:**
- Golden test set (100 hand-labeled examples) → run on every model update
- Shadow deployment → new model runs in parallel, compare outputs
- Gradual rollout (5% → 20% → 100%)

---

### **Quick Recall Bullets for Part 1:**

✅ **RAG vs Fine-tuning:** RAG for changing knowledge + citations, FT for style + cost at scale  
✅ **RAG debug:** Check retrieval → chunk quality → prompt strength → context position  
✅ **Model choice:** API first, open-source at $5K+/month  
✅ **LoRA:** Use for <10K data, multi-task, fast iteration  
✅ **Eval:** Automated (LLM judge) + Human spot-check + Production metrics  
✅ **Loss ≠ Quality:** Always validate on real production distribution  

---



# PART 2

🟠 PART 2: APPLIED SYSTEM THINKING
🟡 PART 2: OPTIMIZATION & SCALING

## 🟠 PART 2: APPLIED SYSTEM THINKING


### **5. Performance & Cost Tradeoffs**

#### **"Latency doubled after launch"**

**Systematic debug approach:**

1. **Add logging at each stage:**
   ```
   Retrieval: 200ms
   Embedding: 50ms
   LLM call: 2500ms ← culprit
   Post-processing: 100ms
   ```

2. **Common causes by component:**
   - **Retrieval slow:** Vector DB overloaded, missing index, cold start
   - **LLM slow:** Context too long, batch size = 1, no KV cache
   - **Network:** API rate limits, timeout retries adding up

3. **Quick wins:**
   - Enable streaming (perceived latency drops)
   - Cache embeddings
   - Reduce max_tokens if over-generating
   - Parallel retrieval + embedding if possible

#### **"Tokens per request keep increasing"**

**Root causes:**
- Conversation history grows unbounded
- Users copy-pasting large documents
- System prompts bloating over time

**Fixes:**
1. **Sliding window:** Keep only last 5 messages in context
2. **Summarization:** Compress old history (every 10 turns → summarize)
3. **Prompt pruning:** Remove redundant instructions
4. **Hard limits:** Cap user input at 2K tokens, context at 8K tokens

#### **"Reduce inference cost without retraining"**

**Immediate actions (sorted by impact):**

1. **Switch to cheaper model for simple queries** (70% of queries don't need GPT-4)
   - Use classifier: GPT-3.5 for routing → GPT-4 only if needed
2. **Prompt compression** (LLMLingua, remove filler words)
3. **Quantization** (16bit → 8bit, 50% cost cut, minimal quality loss)
4. **Reduce temperature** (less sampling = faster, cheaper)
5. **Cache aggressively** (embed common questions, cache similar queries)

#### **"Batch vs Streaming tradeoffs"**

| Aspect | Batch | Streaming |
|--------|-------|-----------|
| **Latency** | High (wait for full response) | Low (perceived, shows tokens immediately) |
| **Throughput** | Higher (GPU efficient) | Lower (harder to batch) |
| **Cost** | Cheaper (better GPU util) | More expensive |
| **UX** | Feels slow | Feels fast |
| **Use case** | Background jobs, bulk processing | Chat, interactive |

**Startup rule:** Streaming for user-facing chat, batch for backend processing

---

### **6. Failure Modes & Debugging**

#### **"Model hallucinates despite correct context"**

**Why it happens:**
- Model's parametric memory conflicts with context
- Context buried in middle of long prompt
- Weak instruction adherence

**Fixes (in order of effort):**
1. **Stronger system prompt:**
   ```
   "You MUST only use information from the context below.
   If the answer isn't in the context, say 'I don't have that information.'"
   ```
2. **Few-shot examples** showing correct refusals
3. **Post-processing filter** (another LLM checks if answer uses context)
4. **Fine-tune** on examples with correct refusal behavior

#### **"Model ignores system prompt occasionally"**

**Common pattern:** Works 90% of time, fails randomly

**Causes:**
- System prompt too long (model loses focus)
- Conflicting instructions (system says X, user says Y)
- Model trained to be helpful > follow rules

**Fixes:**
1. **Shorten system prompt** (80% of instructions are redundant)
2. **Repeat critical rules** at start AND end of prompt
3. **Use delimiters:**
   ```
   ### CRITICAL RULE ###
   Never make up information
   ### END CRITICAL RULE ###
   ```
4. **Switch model** (Claude > GPT for instruction-following)

#### **"Answers degrade for long conversations"**

**Why:**
- Context window fills up → early messages truncated
- Attention dilutes over long sequences
- Contradictory info accumulates

**Fixes:**
1. **Summarize every N turns** (keep intent, drop verbosity)
2. **Prune low-importance turns** (greetings, confirmations)
3. **Reset with context transfer:**
   ```
   After 10 turns: "Here's what we discussed: [summary]. Continue from here."
   ```

#### **"Works in tests, fails in production"**

**The distribution shift problem:**

**Common mismatches:**
- **Test:** Clean, well-formed queries
- **Prod:** Typos, slang, vague questions, multi-intent

**Debug process:**
1. **Log 100 production failures** → manually categorize
2. **Build adversarial test set** from real failures
3. **Add data augmentation** (typos, paraphrases) to eval

**Prevention:**
- Always test on real user data before launch
- Shadow mode for 1 week minimum

---

### **7. Prompting vs Training**

#### **"When is prompting enough?"**

**Prompting works when:**
- Task is clear and well-defined
- Model already knows the domain
- You need fast iteration
- < 100 examples available
- Cost isn't the bottleneck

**Examples:** Summarization, Q&A on public knowledge, code generation

#### **"When does prompt engineering break down?"**

**Red flags prompting won't scale:**
1. **Prompt > 3000 tokens** (context stuffing)
2. **Need 10+ few-shot examples** (just fine-tune)
3. **Desired behavior contradicts base model** (e.g., make Llama give very terse answers)
4. **Cost > $1K/month** on prompts alone

**Migration trigger:** If spending 5+ hours/week tweaking prompts → fine-tune

#### **"How to version and test prompts?"**

**Simple system (enough for startups):**

```python
prompts/
  v1_baseline.txt
  v2_add_examples.txt (A/B test vs v1)
  v3_shorter.txt

# Git commit message:
"Prompt v2: Added 3 examples for edge cases, reduced hallucination by 15%"
```

**Testing:**
1. **Golden set:** 50-100 hand-labeled examples
2. **Automated eval:** LLM-as-judge scores both versions
3. **A/B test:** 10% traffic for 24 hours
4. **Decision rule:** Win on both automated + human eval

#### **"Migrate from prompts → fine-tuning"**

**When to switch:**
- Prompt cost > $500/month
- Latency matters (shorter prompts = faster)
- Need consistent style/format
- Have 500+ good examples

**How to prepare:**
1. **Log best prompt outputs** → becomes training data
2. **Extract system prompt patterns** → bake into fine-tuning
3. **Keep simple prompt post-FT** (you still need some instruction)

**Caution:** Don't fine-tune too early, prompts are more flexible

---

### **8. Data & Drift**

#### **"Data distribution changes silently — how to detect?"**

**Monitoring system (minimum viable):**

1. **Input drift:**
   - Track input length distribution (weekly)
   - Monitor top-K query patterns (clustering)
   - Alert if new query types > 20% of traffic

2. **Output drift:**
   - Track average output length
   - Monitor refusal rate (sudden spike = problem)
   - LLM-as-judge quality score (weekly batch)

3. **User feedback:**
   - Thumbs down rate
   - Session abandonment

**Alert triggers:**
- Refusal rate changes > 10%
- Average quality score drops > 0.3
- Thumbs down increases > 15%

#### **"Retrieval drift vs Model drift"**

| Type | What changed | How to detect | Fix |
|------|--------------|---------------|-----|
| **Retrieval drift** | Knowledge base updated, embeddings stale | Retrieval quality drops, same model | Re-embed documents |
| **Model drift** | User language shifts, model outdated | Retrieval fine, outputs feel off | Fine-tune or update base model |

**Real example:**
- Company rebrands product name
- Old docs still use old name
- Retrieval fails → **Retrieval drift** (update docs)

#### **"User behavior shifts — model correct but useless"**

**Scenario:** Users start asking advanced questions, but model gives beginner-level answers

**Why metrics don't catch it:**
- Answers are still factually correct
- Automated eval passes
- But users are frustrated (not what they need)

**Detection:**
- **Qualitative feedback:** "Too basic", "I know this already"
- **Engagement drops:** Session length decreases
- **Query reformulation:** Users keep asking follow-ups

**Fix:**
- Segment users (beginner vs advanced)
- Adapt system prompt based on user segment
- Add complexity level to prompt: "Assume expert-level knowledge"

---

### **Quick Recall Bullets for Part 2:**

✅ **Latency debug:** Log each component → find bottleneck → cache/parallelize/optimize  
✅ **Token cost:** Sliding window, summarize history, hard caps, route to cheaper models  
✅ **Hallucination:** Stronger instruction + few-shot + context position  
✅ **System prompt ignored:** Too long, conflicting rules → shorten + repeat critical parts  
✅ **Prompt → FT:** Switch when cost > $500/mo OR latency critical OR have 500+ examples  
✅ **Drift detection:** Monitor input/output distributions, refusal rate, quality scores  
✅ **Test ≠ Prod:** Always log real failures, build adversarial test set from production  

---

## 🟡 PART 2: OPTIMIZATION & SCALING



### **9. Inference Optimization Scenarios**

#### **"Why does latency grow with conversation length?"**

**Root cause: Context window processing**

Every turn, the model reprocesses the entire conversation:
```
Turn 1: Process 100 tokens → 200ms
Turn 5: Process 500 tokens → 1000ms
Turn 10: Process 1000 tokens → 2000ms
```

**Why it's quadratic-ish:**
- Self-attention is O(n²) in sequence length
- Each new token attends to ALL previous tokens
- No memory of previous computations (naive implementation)

**Solutions:**

1. **KV-cache (most important):**
   - Cache key/value matrices from previous turns
   - Only compute attention for new tokens
   - Reduces Turn 10 from 2000ms → 250ms
   - **Tradeoff:** Uses more memory (store KV for all tokens)

2. **Conversation compression:**
   - Summarize every 5 turns
   - Keep last 3 messages verbatim, summarize older ones

3. **Stateful sessions:**
   - Store conversation state on server
   - Don't re-send full history every time

#### **"How does KV-cache help and when does it fail?"**

**How it works:**
```
Without KV-cache:
Turn 1: Compute attention for tokens [1-100]
Turn 2: Recompute attention for tokens [1-200] ← wasteful

With KV-cache:
Turn 1: Compute + cache K,V for tokens [1-100]
Turn 2: Reuse cached [1-100], compute only [101-200] ← 2x faster
```

**When KV-cache fails:**

1. **Memory overflow:**
   - Long conversations fill GPU memory
   - Cache size grows linearly with context
   - **Fix:** Evict old KV pairs (sliding window)

2. **Batch processing:**
   - Different conversations have different lengths
   - Can't batch efficiently (padding wastes compute)
   - **Fix:** PagedAttention (vLLM), dynamic batching

3. **Context modification:**
   - If you edit earlier messages, cache is invalid
   - Must recompute from scratch

#### **"vLLM vs HuggingFace generate — when to choose?"**

| Feature | HuggingFace `.generate()` | vLLM |
|---------|---------------------------|------|
| **Ease of use** | Very simple, 3 lines of code | Needs server setup |
| **Throughput** | Low (naive batching) | High (PagedAttention, continuous batching) |
| **Latency (single)** | Same | Same |
| **Latency (concurrent)** | Poor (queue waits) | Good (batches efficiently) |
| **Memory efficiency** | Moderate | High (dynamic KV allocation) |
| **Use case** | Prototyping, low traffic | Production, high traffic |

**Startup decision rule:**
- **<10 req/min:** HuggingFace is fine
- **>50 req/min:** Use vLLM or similar (TGI, TensorRT-LLM)

**Why vLLM wins at scale:**
- **PagedAttention:** KV-cache stored in non-contiguous memory (like OS virtual memory)
- **Continuous batching:** New requests join ongoing batches without waiting
- Result: 2-3x higher throughput, 50% lower latency under load

#### **"How do you handle bursty traffic?"**

**Problem:**
- Normal: 10 req/min
- Peak: 200 req/min (product launch, viral moment)
- Infrastructure sized for normal → peak = downtime

**Solutions (by sophistication):**

1. **Quick fix: Queue + backpressure**
   ```
   if queue_length > 50:
       return "High traffic, please retry in 30s"
   ```
   - Ugly but works for rare spikes

2. **Auto-scaling (cloud):**
   - Spin up more GPU instances on demand
   - **Issue:** Cold start = 2-5 minutes (can't help sudden spikes)
   - **Hybrid:** Keep 1-2 warm standby instances

3. **Request prioritization:**
   - Paying users → high priority queue
   - Free tier → low priority, shed load if needed

4. **Model cascade:**
   ```
   High traffic → route simple queries to smaller/faster model
   Complex queries → still use best model
   ```

5. **Rate limiting (preventive):**
   - Per-user limits (10 req/min)
   - Exponential backoff for repeated requests

**Startup MVP:** Queue + simple rate limiting, monitor 95th percentile latency

---

### **10. Quantization & Deployment**

#### **"Quantized model → accuracy dropped. Why?"**

**What quantization does:**
```
FP16: 16 bits per parameter
INT8: 8 bits per parameter → 2x smaller, 2x faster
INT4: 4 bits → 4x smaller, but risky
```

**Common failure modes:**

1. **Outlier features:**
   - Some activations are 100x larger than others
   - Quantizing them loses critical info
   - **Fix:** Mixed precision (keep outliers in FP16)

2. **Calibration data mismatch:**
   - Quantization uses sample data to set scale factors
   - If calibration data ≠ production data → poor performance
   - **Fix:** Calibrate on real production samples

3. **Too aggressive (INT4 on small models):**
   - < 7B params models are sensitive to INT4
   - **Rule:** FP16 for < 3B, INT8 for 3-13B, INT4 only for 70B+

4. **Wrong quantization method:**
   - Per-tensor quantization (simple, lossy)
   - Per-channel quantization (better, standard)
   - Group-wise quantization (best, GPTQ/AWQ)

#### **"GPTQ vs AWQ — what to choose?"**

| Method | GPTQ | AWQ |
|--------|------|-----|
| **Calibration time** | Slow (hours for 70B) | Fast (minutes) |
| **Quality (INT4)** | Good | Better |
| **How it works** | Minimize quantization error globally | Protect important weights (activation-aware) |
| **Best for** | General use, batch processing | Low-latency serving |

**Startup rule:**
- Default: **AWQ** (faster calibration, slightly better quality)
- Use GPTQ if: Already have infrastructure for it

**Both require:**
- GPU for inference (quantized models still need GPU)
- Proper kernel support (AutoGPTQ, AutoAWQ libraries)

#### **"How to A/B test quantized models safely?"**

**Challenge:** Quality degradation is subtle, not binary

**Safe rollout process:**

1. **Shadow mode (1 week):**
   - Run quantized model in parallel
   - Log outputs, don't serve to users
   - Compare: exact match rate, LLM-as-judge similarity score

2. **Canary (5% traffic):**
   - Serve to 5% of users
   - Monitor: thumbs down rate, session length, refusal rate
   - **Kill switch:** Auto-rollback if metrics degrade >10%

3. **Gradual ramp (5% → 20% → 50% → 100%):**
   - Each stage lasts 2-3 days
   - Pause if any metric regresses

**Metrics to watch:**
- Task success rate (primary)
- User satisfaction (thumbs up/down)
- Refusal rate (quantization can make model refuse more)
- Output length (sometimes drops, bad sign)

**Red flags → immediate rollback:**
- Refusal rate increases >20%
- Thumbs down increases >15%
- Silent failures (empty outputs, broken JSON)

---

### **11. Model Updates & Versioning**

#### **"How do you roll out a new model version?"**

**Standard process (low-risk):**

1. **Offline validation:**
   - Golden test set (100 examples) → must match or beat old model
   - Edge case set (adversarial examples) → no new failures
   - Cost/latency benchmark

2. **Shadow deployment (3-7 days):**
   ```
   User query → Old model (serve this)
                ↓
              New model (log only, compare)
   ```
   - Compare outputs: agreement rate, quality scores
   - Look for: new failure modes, unexpected behaviors

3. **Canary (5% for 2-3 days):**
   - Serve new model to 5% of users
   - Monitor real-time metrics
   - **Decision:** If metrics stable or better → continue
   - **Rollback trigger:** Any metric regresses >10%

4. **Gradual rollout:**
   - 5% → 20% → 50% → 100%
   - Each stage: wait 48 hours, check metrics
   - Keep old model warm (can rollback instantly)

5. **Full deployment:**
   - Deprecate old model after 1 week of stability
   - Keep old model archived for 1 month (just in case)

#### **"How do you rollback quickly?"**

**Pre-rollout setup:**

```python
# Feature flag system
if feature_flag.get("model_version") == "v2":
    model = load_model_v2()
else:
    model = load_model_v1()  # fallback
```

**Instant rollback (< 1 minute):**
- Flip feature flag
- No redeployment needed
- Old model still running in background

**Infrastructure requirements:**
- Keep old model loaded in memory (costs extra RAM)
- OR: Accept 30s cold-start on rollback
- Load balancer can switch traffic instantly

**When to rollback:**
- Error rate spike
- User complaints surge
- Metrics drop significantly
- Silent failures detected

**Startup mistake to avoid:**
- Don't shut down old model immediately after new model launches
- Keep both running for 48 hours minimum

#### **"How do you compare two LLMs fairly?"**

**The challenge:** No single "correct" answer

**Multi-dimensional evaluation:**

1. **Task success (objective):**
   - Retrieval: Did it find right document?
   - Extraction: Did it extract correct entities?
   - Classification: Did it classify correctly?

2. **Quality (subjective but measurable):**
   - **LLM-as-judge (GPT-4):**
     ```
     "Compare these two answers. Rate 1-5 on:
     - Accuracy
     - Helpfulness
     - Conciseness"
     ```
   - Run on 200 diverse examples
   - Calculate win rate (Model A better, B better, tie)

3. **Pairwise human eval:**
   - Show annotators both outputs (blind labels)
   - "Which answer is better?" → A, B, or Tie
   - Need ~100 comparisons for statistical significance
   - **Inter-annotator agreement** must be >70%

4. **Production metrics (most important):**
   - User engagement (thumbs, session length)
   - Task completion rate
   - Cost per task
   - Latency

**Fair comparison checklist:**
- ✅ Same prompts, same retrieval, same system message
- ✅ Same temperature and generation config
- ✅ Test on representative sample (not cherry-picked)
- ✅ Measure multiple metrics (not just one)
- ✅ Statistical significance test (bootstrap, t-test)

**Reporting:**
```
Model A vs Model B:
- Task success: 85% vs 87% (+2%, p=0.04) ✓ significant
- LLM-judge: 4.2 vs 4.3 (+0.1, p=0.23) ✗ not significant
- Latency: 1.2s vs 1.8s (+50%) ✗ worse
- Cost: $0.02 vs $0.015 (-25%) ✓ cheaper

Decision: Keep Model A (faster, slightly worse quality not worth cost)
```

---

### **Quick Recall Bullets for Part 3:**

✅ **Latency + context length:** KV-cache is critical, reduces recomputation  
✅ **vLLM vs HF:** vLLM for >50 req/min, HF for prototyping  
✅ **Bursty traffic:** Queue + rate limiting + auto-scale, monitor P95 latency  
✅ **Quantization:** INT8 safe for 7B+, INT4 only for 70B+, calibrate on prod data  
✅ **GPTQ vs AWQ:** AWQ faster + slightly better, default choice  
✅ **Model rollout:** Shadow → Canary (5%) → Gradual (20/50/100) → Keep old model warm  
✅ **Rollback:** Feature flags, keep old model running 48h, instant switch  
✅ **LLM comparison:** LLM-as-judge + human pairwise + production metrics, need statistical significance  

---

# 🟢 P3 ADVANCED / DIFFERENTIATOR


### **12. Research-Flavored Scenarios**

#### **"How would you test a new attention mechanism?"**

**Context:** Paper claims "Sparse Attention improves efficiency with no quality loss"

**Validation process (startup-practical):**

1. **Reproducibility check (1-2 days):**
   - Can you actually run their code?
   - Do their numbers match on standard benchmarks?
   - **Red flag:** If you can't reproduce → skip it

2. **Your domain test (3-5 days):**
   ```
   Test on YOUR actual use case, not paper's benchmarks
   
   Example: Customer support chatbot
   - Run 500 real production queries
   - Measure: quality (LLM-judge), latency, memory
   ```

3. **Edge cases (2 days):**
   - Very long contexts (paper tested 4K, you need 16K)
   - Diverse query types (paper used Q&A, you have multi-turn chat)
   - Failure modes (does it break unexpectedly?)

4. **Engineering cost (critical):**
   - Implementation time: 1 day vs 2 weeks?
   - Maintenance burden: custom CUDA kernels vs drop-in replacement?
   - Team expertise: Do you have people who can debug this?

**Decision framework:**
```
Ship if:
✓ 10%+ quality improvement OR 30%+ speedup
✓ Works on your data (not just paper's)
✓ Implementation cost < 1 week
✓ No new critical failure modes

Skip if:
✗ Marginal gains (<5% on your data)
✗ Requires custom infrastructure
✗ Team can't maintain it
```

**Startup reality:** Most research optimizations aren't worth it unless gains are huge (>20%)

#### **"How do you decide if a paper is production-worthy?"**

**Quick filter (read in 15 mins):**

1. **Code available?**
   - No code = not production-ready
   - Code but no pretrained weights = rebuild cost too high

2. **Realistic evaluation?**
   - Paper tested on academic benchmarks only → skeptical
   - Paper includes production scenarios, latency, cost → promising

3. **Comparison to strong baselines?**
   - "Beats GPT-2" (2019 model) → not useful
   - "Beats GPT-4 on X" → interesting

4. **Clear failure modes discussed?**
   - Paper only shows successes → likely cherry-picked
   - Paper discusses when it fails → more honest

**Deep evaluation (if passes filter):**

5. **Test on YOUR specific problem:**
   - Don't trust benchmarks blindly
   - Run 100-200 examples from your domain
   - Measure what matters: task success, latency, cost

6. **Integration cost estimate:**
   - Drop-in replacement (change model name) → 1 day
   - New architecture (custom code) → 1-2 weeks
   - Custom training pipeline → 1 month+

7. **Maintenance cost:**
   - Will this break with library updates?
   - Does team have expertise to debug?
   - Is there community support?

**Real examples:**

✅ **Worth it:** FlashAttention (2022)
- Massive speedup (2-4x)
- Drop-in replacement
- Widely adopted, good support

❌ **Not worth it:** Most "novel architecture" papers
- Marginal gains
- Requires custom implementation
- No pretrained weights
- Team can't maintain

#### **"How do you validate small improvements statistically?"**

**Problem:** Claim "3% improvement" but is it real or noise?

**Statistical validation:**

1. **Sufficient sample size:**
   ```
   For detecting 3% improvement with 95% confidence:
   Need ~1000-1500 samples (depends on variance)
   
   Too small: 50 samples → can't trust 3% difference
   ```

2. **Bootstrap confidence intervals:**
   ```python
   # Resample your test set 1000 times
   # Calculate metric each time
   # 95% CI: [2.5th percentile, 97.5th percentile]
   
   If improvement CI = [1.5%, 4.5%] → significant
   If improvement CI = [-0.5%, 6.5%] → not significant (crosses 0)
   ```

3. **Paired testing (critical):**
   - Test both models on SAME examples
   - Reduces variance, increases statistical power
   - Wrong: Model A on 500 samples, Model B on different 500
   - Right: Both models on same 500 samples

4. **Multiple metrics:**
   ```
   Don't just report one metric where you won
   
   Report:
   - Primary metric (task success)
   - Quality metrics (relevance, coherence)
   - Efficiency metrics (latency, cost)
   - Failure modes (refusal rate, hallucination)
   ```

5. **Significance testing:**
   - **T-test** (if metrics are normally distributed)
   - **Mann-Whitney U** (if not normal, e.g., ratings 1-5)
   - **p-value < 0.05** = significant
   - But also check **effect size** (Cohen's d)

**Red flags:**
- Only tested on 50 examples → too small
- Only reports improvement on 1 cherry-picked metric
- No confidence intervals or p-values
- "Improvement" smaller than measurement noise

**Practical rule:**
```
Claim 3% improvement as real only if:
✓ Tested on 1000+ samples
✓ p-value < 0.05
✓ Confidence interval doesn't include 0
✓ Consistent across multiple metrics
✓ Holds on diverse subsets of data
```

---

### **13. Multi-Agent & Tooling**

#### **"When are agents overkill?"**

**Use single LLM call when:**
- Task is simple, one-step (Q&A, summarization)
- Latency critical (< 1s response needed)
- Limited budget (each agent step costs $$$)
- Failure modes hard to debug

**Use agents when:**
- Task requires multiple steps (search → analyze → format)
- Steps are conditional (if X then do Y, else Z)
- Need external tools (search, calculator, database)
- Correctness > latency

**Real examples:**

❌ **Agent overkill:**
- Simple customer support FAQ
- Basic document summarization
- Single-step classification

✅ **Agents make sense:**
- Complex research tasks (search multiple sources → synthesize)
- Data analysis (query DB → analyze → visualize)
- Multi-step workflows (validate input → process → verify output)

**Cost comparison:**
```
Single call: $0.01
Agent (3 steps): $0.03-0.05
Agent (10 steps): $0.10-0.20

If 80% of queries can be solved in 1 call → don't use agent
```

#### **"Design a tool-calling system safely"**

**Core safety challenges:**
1. Model calls wrong tool
2. Model generates invalid parameters
3. Model stuck in infinite loop
4. Tool returns error, model doesn't handle it

**Safe design (layered defense):**

**Layer 1: Constrain tool choice**
```python
# Don't give model all tools at once
# Give only relevant tools based on query

if "weather" in query:
    tools = [weather_api]
elif "calculation" in query:
    tools = [calculator]
else:
    tools = [search]  # safe default
```

**Layer 2: Validate parameters**
```python
def safe_tool_call(tool, params):
    # Validate before calling
    if tool == "database_query":
        if not validate_sql(params["query"]):
            return "Invalid SQL, please retry"
        if is_destructive(params["query"]):  # DELETE, DROP
            return "Destructive queries not allowed"
    
    # Sandbox execution
    try:
        result = tool.execute(params, timeout=5)
    except TimeoutError:
        return "Tool execution timeout"
    
    return result
```

**Layer 3: Limit iterations**
```python
MAX_STEPS = 5

for step in range(MAX_STEPS):
    action = model.generate(prompt)
    if action == "FINAL_ANSWER":
        break
    result = safe_tool_call(action.tool, action.params)
    prompt += result

if step == MAX_STEPS - 1:
    return "Could not complete task in allowed steps"
```

**Layer 4: Human-in-the-loop for risky actions**
```python
RISKY_TOOLS = ["send_email", "charge_payment", "delete_data"]

if action.tool in RISKY_TOOLS:
    # Show user confirmation
    return f"About to {action.tool} with {params}. Confirm?"
```

**Layer 5: Audit logging**
```python
# Log every tool call
log({
    "timestamp": now(),
    "user_id": user_id,
    "query": query,
    "tool": tool_name,
    "params": params,
    "result": result,
    "model_reasoning": reasoning
})
```

#### **"How do you prevent tool hallucinations?"**

**What is tool hallucination?**
```
User: "What's the weather in Paris?"
Model: "I'll call weather_api(city='Paris')"
Model: [fabricates] "The API returned: 72°F and sunny"
       ↑ Model made this up, didn't actually call API
```

**Prevention strategies:**

1. **Structured output enforcement:**
```python
# Force model to output JSON, parse strictly
response = model.generate(
    prompt,
    response_format={"type": "json_object"}
)
tool_call = json.loads(response)  # Fails if not valid JSON

# Then YOU execute the tool, model doesn't report results
actual_result = execute_tool(tool_call)
```

2. **Separate generation from execution:**
```python
# Step 1: Model generates plan
plan = model.generate("What tools do you need?")

# Step 2: YOU execute tools
results = []
for tool in plan.tools:
    result = actually_call_api(tool)
    results.append(result)

# Step 3: Model synthesizes (with real results injected)
answer = model.generate(f"Based on results: {results}, answer:")
```

3. **Add result verification:**
```python
# Model must reference specific result fields
system_prompt = """
You MUST cite which tool result you're using:
"According to weather_api result, temperature is {result.temp}°F"

Do NOT make up tool results.
"""
```

4. **Prompt design:**
```
Bad prompt:
"Use the weather tool and tell the user the result"
↑ Model might fabricate result

Good prompt:
"First, output: ACTION: weather_api
Then wait for system to provide result.
Then format the result for user."
```

**Detection (post-hoc):**
- Compare model's stated result with actual logged API response
- If mismatch → log as hallucination, retrain with corrected examples

---

### **14. RLHF / Alignment (High-Level)**

#### **"Why is PPO expensive in practice?"**

**PPO = Proximal Policy Optimization (used in ChatGPT training)**

**Cost breakdown:**

1. **Need 4 models loaded simultaneously:**
   ```
   - Policy model (being trained)        → 70B params
   - Reference model (frozen baseline)   → 70B params
   - Reward model (scoring outputs)      → 7B params
   - Value model (for advantages)        → 7B params
   
   Total: ~220B params in GPU memory
   Requires: 8x A100 GPUs minimum
   Cost: $20-30/hour
   ```

2. **Sample inefficiency:**
   - Generate responses → score them → update policy
   - Each update uses only 1 batch of data (can't reuse)
   - Need millions of samples → weeks of training
   - Cost: $50K-200K for full RLHF run

3. **Reward model training:**
   - Need 10K-50K human preference labels
   - Labeling cost: $0.50-2 per comparison
   - Total: $5K-100K just for labels

4. **Hyperparameter sensitivity:**
   - PPO is finicky, needs careful tuning
   - Many failed runs before finding good config
   - Multiply costs by 3-5x for experimentation

**Why startups can't afford it:**
- Needs dedicated ML infra team
- Requires 100K+ training budget
- 1-2 month timeline
- High risk of failure

#### **"Why startups prefer DPO?"**

**DPO = Direct Preference Optimization**

**Why it's cheaper:**

1. **Only 1 model needed:**
   ```
   PPO: 4 models (220B params)
   DPO: 1 model (70B params)
   
   Memory: 4x less
   Cost: 4x cheaper
   ```

2. **Simpler training:**
   - No reward model to train separately
   - No RL instability issues
   - Standard supervised learning pipeline
   - Easier to debug

3. **Data efficient:**
   - Can reuse preference data multiple epochs
   - Needs fewer samples than PPO
   - 5K-10K preferences often enough (vs 50K+ for PPO)

4. **Faster iteration:**
   ```
   PPO: 2-4 weeks per run
   DPO: 2-3 days per run
   
   More experiments in same budget
   ```

**Quality comparison:**
- DPO often matches PPO quality
- Slightly less flexible (can't use complex reward functions)
- But for most tasks, good enough

**Startup default:** Start with DPO, only consider PPO if:
- Have >$100K budget
- Need very specific reward shaping
- Have ML research team

#### **"When is RLHF not worth it?"**

**Skip RLHF when:**

1. **Small data regime:**
   - Have < 1000 preference labels
   - RLHF needs scale to work
   - Better: Few-shot prompting or small SFT

2. **Clear objective function:**
   ```
   Have: Exact match, F1, ROUGE (can compute automatically)
   Don't need: Human preferences
   
   Just do supervised fine-tuning on correct outputs
   ```

3. **Rapid iteration needed:**
   - RLHF takes weeks
   - If need to ship in days → use prompting

4. **Budget <$10K:**
   - Can't afford proper RLHF run
   - Use synthetic preferences instead (LLM-as-judge)

5. **Base model already good:**
   ```
   GPT-4 / Claude are already RLHF'd
   Adding your own RLHF might hurt more than help
   
   Better: Prompt engineering or small task-specific fine-tune
   ```

**When RLHF IS worth it:**
- Have 10K+ human preferences
- Subjective quality matters (helpfulness, tone)
- Budget >$50K
- Need model to learn nuanced behavior (what humans actually prefer vs what's "correct")
- Have time (1-2 months)

**Middle ground for startups:**
```
Synthetic RLHF:
1. Generate responses from base model
2. Use GPT-4 to rank them (simulated preferences)
3. Train with DPO on synthetic preferences

Cost: $500-2K (100x cheaper)
Quality: 70-80% of real RLHF
Time: 1 week
```

---

### **15. Product Sense (Rare but Powerful)**

#### **"When should you remove an LLM feature?"**

**Kill criteria (any one is enough):**

1. **Low usage:**
   - < 5% of users use it
   - Those who use it, use it < 2x/month
   - Indicates: Not solving real problem

2. **High support burden:**
   - Generates more support tickets than value
   - Users constantly confused about how it works
   - Cost to support > revenue/value generated

3. **Quality ceiling hit:**
   - Can't get accuracy above 70%
   - Users frustrated by errors
   - No clear path to improvement
   - Example: Complex math reasoning, current LLMs not good enough

4. **Cost unsustainable:**
   ```
   Monthly feature cost: $5K
   Feature revenue: $1K
   Burn rate: -$4K/month
   
   Unless strategic → kill it
   ```

5. **Trust erosion:**
   - Feature hallucinated once → users lost trust
   - Now they don't use ANY LLM features
   - One bad feature poisoning the well

6. **Better alternatives exist:**
   - Users use competitor's feature instead
   - Or: Non-LLM solution works better
   - Example: Template-based email > LLM-generated

**How to make the decision:**
```
1. Data-driven:
   - Usage metrics (DAU, frequency)
   - Quality metrics (success rate, user satisfaction)
   - Cost per successful interaction

2. User research:
   - Why aren't people using it?
   - What do they use instead?
   - Would they miss it if removed?

3. Strategic value:
   - Does it differentiate us?
   - Is it a platform play? (lose money now, strategic later)
   - Does it attract users even if not used?

Decision: Kill if metrics bad AND no strategic value
```

**How to sunset gracefully:**
- Announce 1 month in advance
- Offer migration path (export data, alternative features)
- Gather feedback (maybe you misunderstood the problem)

#### **"How do you explain model limitations to PMs?"**

**Bad approach:**
"LLMs hallucinate, it's just how they work"
→ PM hears: "You can't build reliable products"

**Good approach: Translate to product constraints**

```
PM asks: "Can we build automated customer support?"

You say:
"Yes, with guardrails. Here's what's realistic:

✓ Can do (>90% accuracy):
  - Answer FAQ questions
  - Route tickets to right team
  - Summarize long conversations

⚠️ Needs review (70-80% accuracy):
  - Handle complex multi-step issues
  - Interpret ambiguous requests
  - Handle edge cases

✗ Can't do reliably (<60% accuracy):
  - Financial calculations
  - Legal advice
  - Guarantee 100% factual accuracy

Trade-offs:
- Option A: Human-in-the-loop (slower, accurate)
- Option B: Fully automated (fast, 10% error rate)
- Option C: Hybrid (auto for simple, human for complex)

Which aligns with our product goals?"
```

**Framework for PM conversations:**

1. **Frame in business terms:**
   - Not: "Attention mechanism limitations"
   - But: "Works for 80% of queries, need fallback for 20%"

2. **Quantify risks:**
   - "95% accuracy means 1 in 20 users see bad output"
   - "At 10K users/day, that's 500 bad experiences"

3. **Offer solutions, not just problems:**
   - "We can't do X perfectly, but here are 3 approaches with different trade-offs"

4. **Set realistic expectations early:**
   - "This will feel like ChatGPT sometimes, not Google"
   - "Users will need to verify outputs"

5. **Show what good looks like:**
   - Demo on real examples
   - Show failure cases too (manage expectations)

#### **"How do you handle user trust after hallucinations?"**

**Scenario:** Your LLM feature gave wrong medical/financial advice, users are upset

**Immediate response (24 hours):**

1. **Acknowledge publicly:**
   ```
   "We're aware Feature X provided incorrect information.
   We've temporarily disabled it while we investigate.
   We take this seriously."
   ```
   
2. **Immediate safety measures:**
   - Add disclaimer: "Verify critical information"
   - Add confidence scores (if low, show warning)
   - Human review for high-stakes domains

3. **Root cause:**
   - Was it retrieval failure? (wrong context)
   - Model hallucination? (fabricated facts)
   - Edge case? (input type never tested)

**Medium-term (1-2 weeks):**

4. **Product changes:**
   ```
   Before: Direct answer
   After: Answer + sources + confidence + "Verify if critical"
   
   Before: Auto-execute actions
   After: Show preview, require confirmation
   ```

5. **Evaluation upgrade:**
   - Add adversarial test cases
   - Red-team the feature (try to break it)
   - Add monitoring for high-stakes queries

6. **Communication:**
   - Explain what went wrong (transparently)
   - What you changed
   - How you're preventing it

**Long-term (ongoing):**

7. **Rebuild trust:**
   - Show accuracy metrics publicly
   - User controls (toggle features on/off)
   - Easy reporting (thumbs down, "this is wrong" button)
   - Show you're taking feedback seriously

8. **Product positioning:**
   - Not: "AI that knows everything"
   - But: "AI assistant that helps, but you stay in control"

9. **Domain-specific boundaries:**
   ```
   Medical: "Not medical advice, consult doctor"
   Legal: "Not legal advice, consult lawyer"
   Financial: "Not financial advice, DYOR"
   
   + Technical measures (refuse certain queries)
   ```

**What NOT to do:**
- ❌ Blame the user ("You should have known")
- ❌ Blame the technology ("All LLMs do this")
- ❌ Over-promise fixes ("Will never happen again")
- ❌ Hide the incident (users remember, trust erodes)

**Key principle:**
"Trust is built slowly, lost quickly, rebuilt even slower"
→ Over-invest in safety for high-stakes domains

---

### **Quick Recall Bullets for Part 4:**

✅ **Research paper validation:** Reproduce → test on YOUR data → measure engineering cost → ship only if >10% gain  
✅ **Statistical significance:** Need 1000+ samples, bootstrap CI, paired testing, p< 0.05  
✅ **Agents:** Overkill for simple tasks, use when multi-step + conditional logic needed  
✅ **Tool safety:** Validate params, limit iterations, sandbox execution, audit log  
✅ **Tool hallucination:** Separate generation from execution, YOU call tools, inject real results  
✅ **PPO vs DPO:** PPO 4x more expensive, needs 4 models; DPO simpler, 1 model, startups prefer it  
✅ **RLHF not worth it:** When < 1K labels, clear objective exists, budget <$10K, rapid iteration needed  
✅ **Kill LLM feature:** Low usage + high cost + quality ceiling + trust erosion  
✅ **Explain to PMs:** Use business terms, quantify risks, offer trade-offs, set realistic expectations  
✅ **Trust after hallucination:** Acknowledge, add safeguards, explain transparently, rebuild slowly  

---

## 🎯 FINAL SUMMARY: TOP 20 MUST-KNOW POINTS

If you remember only these, you'll handle 80% of interviews:

### Design & Architecture
1. **RAG vs Fine-tuning:** RAG for changing knowledge, FT for style/cost at scale
2. **Evaluation 3-tier:** Automated (LLM-judge) → Human spot-check → Production metrics
3. **Model choice:** API first, open-source at scale ($5K+/mo)

### Debugging & Failure Modes
4. **RAG debug order:** Retrieval → chunk quality → prompt strength → context position
5. **Hallucination fix:** Stronger instruction + few-shot + post-filter
6. **Test ≠ Prod:** Always build adversarial test set from real failures

### Performance & Cost
7. **Latency debug:** Log each component → find bottleneck → optimize that
8. **Token cost control:** Sliding window + summarize + hard caps + route to cheaper models
9. **KV-cache:** Critical for multi-turn, saves recomputation, uses more memory

### Optimization
10. **vLLM for production:** >50 req/min needs vLLM/TGI, not HuggingFace
11. **Quantization rules:** INT8 for 7B+, INT4 only for 70B+, calibrate on prod data
12. **Bursty traffic:** Queue + rate limit + auto-scale, monitor P95/P99 latency

### Model Updates
13. **Safe rollout:** Shadow → Canary (5%) → Gradual → Keep old model warm 48h
14. **Rollback:** Feature flags, instant switch, keep old model running
15. **Compare models:** LLM-judge + human pairwise + production metrics + statistical significance

### Advanced Topics
16. **Agents:** Use only for multi-step conditional workflows, overkill for simple tasks
17. **Tool safety:** Validate params, limit iterations, YOU execute tools (prevent hallucination)
18. **DPO vs PPO:** DPO 4x cheaper, simpler, good enough for startups
19. **Research papers:** Test on YOUR data, ship only if >10% gain, consider engineering cost

### Product Thinking
20. **Interview mindset:** Clarify constraints → Simple baseline → Failure modes → Iteration plan

---